# Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score


## From here it's workable

In [2]:
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

In [3]:
# 逐个读取feature_engineered_training_chunk{i}并上下拼接到一个df
base_path = '/Users/eva/Documents/Study/Y1S2/DMT/assignment2/'
file_pattern = 'feature_engineered_training_chunk_{}.csv'
for i in range(10):
    df_chunk = pd.read_csv(base_path + file_pattern.format(i))
    df_chunk['score'] = df_chunk.apply(assign_scores, axis=1)
    if i == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk], axis=0)
df.head()


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp8_rate_percent_diff_prop_id_mean,comp8_rate_percent_diff_prop_id_median,comp8_rate_percent_diff_prop_id_std,comp8_rate_percent_diff_srch_id_mean,comp8_rate_percent_diff_srch_id_median,comp8_rate_percent_diff_srch_id_std,comp8_rate_percent_diff_visitor_location_country_id_mean,comp8_rate_percent_diff_visitor_location_country_id_median,comp8_rate_percent_diff_visitor_location_country_id_std,score
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0
3,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,27348,2,4.0,...,5.0,5.0,NaN,5.0,5.0,NaN,5.0,5.0,NaN,0
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0


In [4]:
# 检查df NA 展示有NA的列
na_counts = df.isna().sum()

# 打印有NA值的列和NA值的数量
print("Columns with NA values and their counts:")
for col, count in na_counts.items():
    if count > 0:
        print(f"{col}: {count}")
# 删除有NA值的列
df = df.dropna(axis=1)


Columns with NA values and their counts:
visitor_hist_starrating_prop_id_std: 4956532
visitor_hist_starrating_srch_id_std: 4956532
visitor_hist_starrating_visitor_location_country_id_std: 4956532
visitor_hist_adr_usd_prop_id_std: 4956532
visitor_hist_adr_usd_srch_id_std: 4956532
visitor_hist_adr_usd_visitor_location_country_id_std: 4956532
prop_starrating_prop_id_std: 4956532
prop_starrating_srch_id_std: 4956532
prop_starrating_visitor_location_country_id_std: 4956532
prop_review_score_prop_id_std: 4956532
prop_review_score_srch_id_std: 4956532
prop_review_score_visitor_location_country_id_std: 4956532
prop_location_score1_prop_id_std: 4956532
prop_location_score1_srch_id_std: 4956532
prop_location_score1_visitor_location_country_id_std: 4956532
prop_location_score2_prop_id_std: 4956532
prop_location_score2_srch_id_std: 4956532
prop_location_score2_visitor_location_country_id_std: 4956532
prop_log_historical_price_prop_id_std: 4956532
prop_log_historical_price_srch_id_std: 4956532
prop

In [5]:
columns = df.columns

# 使用列表推导式筛选出不需要删除的列
features = [
    col for col in columns if col not in ['date_time', 'position', 'click_bool', 'booking_bool', 'score']
    and 'gross_bookings_usd' not in col and 'position' not in col
]


print(len(features))
print(len(df.columns))

188
200


In [8]:
print(features)

['srch_id', 'site_id', 'visitor_location_country_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff', 'bool_visitor_hist', 'visitor_his

### Default Params

In [ ]:
default_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'max_depth': 3,
    'num_leaves': 28,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'min_child_samples': 40,
    'min_child_weight': 0.001,
    'bagging_fraction': 1,
    'bagging_freq': 2,
    'reg_alpha': 0.001,
    'reg_lambda': 8,
    'cat_smooth': 0,
    'num_iterations': 800,
    'is_unbalance': True  # 仅当你确信数据不平衡严重且影响模型性能时才设置
}

In [24]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import random

unique_ids = df['srch_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)  # 首先分割出20%为最终测试集
train_val_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42, shuffle=True)  # 将剩余80%分为训练集(60%)和验证集(20%)

train_df = df[df['srch_id'].isin(train_val_ids)]
val_df = df[df['srch_id'].isin(val_ids)]
test_df = df[df['srch_id'].isin(test_ids)]

train_data = lgb.Dataset(train_df[features], label=train_df['score'], group=train_df['srch_id'].value_counts().sort_index())
val_data = lgb.Dataset(val_df[features], label=val_df['score'], group=val_df['srch_id'].value_counts().sort_index())
test_data = lgb.Dataset(test_df[features], label=test_df['score'], group=test_df['srch_id'].value_counts().sort_index())

# 参数空间
param_grid = {
    'num_leaves': [10, 31, 50, 70],
    'max_depth': [3, 5, 6, 8],
    'min_child_samples': [10, 20, 30, 40],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'feature_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_freq': [1, 2, 4],
    'reg_alpha': [0, 0.001, 0.01, 0.1],
    'reg_lambda': [0, 1, 5, 10]
}

# 随机搜索
best_ndcg = 0
best_params = None
n_iterations = 50  # 定义迭代次数

param_grid = {
    'num_leaves': [10, 31, 50, 70],
    'max_depth': [3, 5, 6, 8],
    'min_child_samples': [10, 20, 30, 40],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'feature_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9],
    'bagging_freq': [1, 2, 4],
    'reg_alpha': [0, 0.001, 0.01, 0.1],
    'reg_lambda': [0, 1, 5, 10]
}

# 随机搜索最佳参数
best_ndcg = 0
best_params = None
n_iterations = 50

for _ in range(n_iterations):
    random_params = {k: random.choice(v) for k, v in param_grid.items()}
    random_params.update({'objective': 'lambdarank', 'metric': 'ndcg', 'ndcg_eval_at': [3, 5], 'verbose': -1, 
                          'boosting_type': 'gbdt', 'num_iterations': 200, 'feature_pre_filter': False})

    bst = lgb.train(
        random_params,
        train_data,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(30)]
    )

    test_pred = bst.predict(test_df[features], num_iteration=bst.best_iteration)
    test_df['predictions'] = test_pred
    test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)
    grouped = test_df.groupby('srch_id')
    ndcg_scores = []

    for name, group in grouped:
        true_relevance = group['score'].values
        scores_pred = group['predictions'].values
        if len(np.unique(true_relevance)) > 1:
            ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

    average_ndcg = np.mean(ndcg_scores)
    print(f"Average NDCG Score: {average_ndcg}")
    if average_ndcg > best_ndcg:
        best_ndcg = average_ndcg
        best_params = random_params

print(f"Best NDCG Score: {best_ndcg}")
print("Best Parameters:", best_params)

/Users/eva/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[127]	valid_0's ndcg@3: 0.320749	valid_0's ndcg@5: 0.377528


/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/1727474269.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions'] = test_pred
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/1727474269.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)
/Users/eva/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in 

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[199]	valid_0's ndcg@3: 0.311923	valid_0's ndcg@5: 0.367889


/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/1727474269.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions'] = test_pred
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/1727474269.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)
/Users/eva/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in 

...

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[170]	valid_0's ndcg@3: 0.320727	valid_0's ndcg@5: 0.376626


/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/1727474269.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions'] = test_pred
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/1727474269.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)
/Users/eva/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in 

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[198]	valid_0's ndcg@3: 0.315798	valid_0's ndcg@5: 0.372237


/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/1727474269.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions'] = test_pred
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/1727474269.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)


Best NDCG Score: 0.3795298612088496
Best Parameters: {'num_leaves': 50, 'max_depth': 8, 'min_child_samples': 40, 'learning_rate': 0.1, 'feature_fraction': 0.6, 'bagging_fraction': 0.7, 'bagging_freq': 4, 'reg_alpha': 0.1, 'reg_lambda': 10, 'objective': 'lambdarank', 'metric': 'ndcg', 'ndcg_eval_at': [3, 5], 'verbose': -1, 'boosting_type': 'gbdt', 'num_iterations': 200, 'feature_pre_filter': False}


### Validation Result Documentation

1. params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [3, 5],
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1,
    'max_depth': 3,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'min_child_samples': 40,
    'min_child_weight': 0.001,
    'bagging_fraction': 1,
    'bagging_freq': 2,
    'reg_alpha': 0.001,
    'reg_lambda': 8,
    'num_iterations': 200,
} 
    0.7177144351777821
2. params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [3, 5],
    'learning_rate': 0.1,
    'num_leaves': 20,
    'verbose': -1,
    'max_depth': 4,
    'learning_rate': 0.1,
    'min_child_samples': 40,
    'min_child_weight': 0.001,
    'bagging_fraction': 1,
    'bagging_freq': 2,
    'reg_alpha': 0.001,
    'reg_lambda': 8,
    'num_iterations': 200,
    'boosting_type': 'gbdt'
}0.7216631446757388
3. params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [3, 5],
    'learning_rate': 0.1,
    'num_leaves': 25,
    'verbose': -1,
    'max_depth': 5,
    'learning_rate': 0.1,
    'min_child_samples': 30,
    'min_child_weight': 0.001,
    'bagging_fraction': 1,
    'bagging_freq': 2,
    'reg_alpha': 0.001,
    'reg_lambda': 8,
    'num_iterations': 200,
    'boosting_type': 'gbdt',
    'feature_fraction': 0.8
}Average NDCG Score: 0.6297555975271502

In [26]:
unique_ids = df['srch_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)
train_df = df[df['srch_id'].isin(train_val_ids)]
test_df = df[df['srch_id'].isin(test_ids)]


# 准备 LightGBM 数据结构
train_data = lgb.Dataset(train_df[features], label=train_df['score'], group=train_df['srch_id'].value_counts().sort_index())
test_data = lgb.Dataset(test_df[features], label=test_df['score'], group=test_df['srch_id'].value_counts().sort_index())

# 设置模型参数
params = {'num_leaves': 50, 'max_depth': 8, 'min_child_samples': 40, 'learning_rate': 0.1, 'feature_fraction': 0.6, 
          'bagging_fraction': 0.7, 'bagging_freq': 4, 'reg_alpha': 0.1, 'reg_lambda': 10, 
          'objective': 'lambdarank', 'metric': 'ndcg', 'ndcg_eval_at': [3, 5], 'verbose': -1, 'boosting_type': 'gbdt', 
          'num_iterations': 200, 'feature_pre_filter': False}

# 训练模型
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# 预测测试集
test_pred = bst.predict(test_df[features])

# 评估模型，计算 NDCG 分数
test_df['predictions'] = test_pred

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


/Users/eva/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/834714577.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions'] = test_pred
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_77189/834714577.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.sort_values(['srch_id', 'predictions'],

Average NDCG Score: 0.3795538974945831


### Result output

In [27]:
# 使用迭代器逐块读取数据
chunk_size = 10000
reader = pd.read_csv('/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_test_set_VU_DM.csv', chunksize=chunk_size)

predictions = []  # 创建一个空列表以存储每个块的预测结果
for chunk in reader:
    # 可以在这里添加数据预处理步骤，比如填充缺失值等
    chunk_pred = bst.predict(chunk[features])  # 应用模型进行预测
    chunk['predictions'] = chunk_pred  # 将预测结果添加到 DataFrame
    predictions.append(chunk[['srch_id', 'prop_id', 'predictions']])  # 仅保留需要的列

# 合并所有批次的预测结果
final_predictions = pd.concat(predictions)

In [28]:
# 确保按照预测分数排序，如果 Kaggle 要求
final_predictions.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 选择需要的列
final_predictions = final_predictions[['srch_id', 'prop_id']]

# 保存为 CSV 文件，确保不包含索引，包含列标题
final_predictions.to_csv('train=all_featured_cleaned(train+set_all_grouped_by).csv', index=False, header=True)

In [29]:
print(final_predictions)

         srch_id  prop_id
23             1    99484
9              1    54937
12             1    61934
5              1    28181
4              1    24194
...          ...      ...
4959177   332787    29018
4959178   332787    32019
4959181   332787    94437
4959182   332787    99509
4959180   332787    35240

[4959183 rows x 2 columns]
